# MARK - Extended

In this project we re-implemented [MARK Model](https://openreview.net/pdf?id=sFyrGPCKQJC) from scratch. We first showed recreated the results on Cifar-100 dataset, and also highlighted possible issues in the paper. We then implemented the model on ImageData, followed by MarketCL dataset. We showed the effectiveness of MARK on such complex datasets. This was followed by incorporating SupSup Framework in order to allow scalability of model to much larger number of tasks, while being efficient in terms of model size. 

In this report, we show a demo training and testing code for MarketCL Model. Relevant flags can be adjusted to run various variations. Whole code is available in the same directory.

# Requirements

!pip install -r requirements.txt

In [ ]:
# Download and PreProcess the datasets
!python prepare_dataset.py

# Imports

In [5]:
import torch
import torch.nn as nn
import numpy as np
from models_marketcl import MARKLSTMModel
from trainer_market import test_loop
from trainer_market import train_fe, train_kb_nonmeta, train_mg_n_clf, update_kb, train_kb_nonmeta_baseline
from dataloaders import get_marketcl_dataloader
from advanced_logger import LogPriority
import pickle
import os
import sys

# Experimenter Class
For Creating Code Backups, Logging and config handling

In [8]:
from experimenter import Experimenter
cfg_file = 'configs/market1.yml'
experimenter = Experimenter(cfg_file)
cfg = experimenter.config
print = experimenter.logger.log

Experiment: MarketExperiment7
Experiment Description: Market Experiment with larger FE SupSup Disabled
Config File: configs/market1.yml
[23:30:25]: Experiment started


# Data Loading

In [9]:
train_dl = get_marketcl_dataloader(batch_size = cfg.TRAINING.BATCH_SIZE) # Train dl
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the Models

In [12]:
model = MARKLSTMModel(cfg, device, 0)
criterion = nn.CrossEntropyLoss().to(device)

False <class 'bool'>


In [13]:

# Main Train Loop
# At the End of the day, it evaluates the model on next days data as well.
# Once training for all symbols on that day is done, testing on all symbols is performed to 
# get backward transfers and final accuracies.
def train(model : MARKLSTMModel, criterion, train_dl, experimenter, device, baseline, iter):


    print = experimenter.logger.log

    # There are four steps:
    # 1. Train the feature extractor on given task.
    # 2. Train KB if task = 0
    # 3. Train MaskGenerator and Classifier
    # 4. Use Meta-Learning to train the KB, and KBClassifier 
    NUM_DAYS = 16
    NUM_SYMBOLS = 20

    days_accs = np.zeros((4, NUM_DAYS-1))
    days_bwt = np.zeros((4, NUM_DAYS - 1))

    NS1=[i for i in range(0, 308)]

    # Only those symbols were chosen, which had data of all days in them
    # Note this is different from filling missing values in original data frame
    # For that, mean for that symbol on that day was used to fill NaN values.
    NS1=list(set(NS1)-set([4, 5, 7, 8, 9, 11, 12, 13, 14, 16, 21, 27, 28, 36, 39, 40, 41, 42, 43, 45, 50, 52, 53, 57, 59, 61, 63, 65, 66, 67, 72, 74, 75, 76, 77, 78, 79, 80, 82, 83, 87, 88, 89, 92, 93, 98, 99, 101, 104, 106, 107, 108, 112, 113, 115, 116, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 154, 157, 160, 162, 163, 165, 166, 168, 171, 173, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 213, 218, 220, 222, 224, 226, 237, 238, 240, 241, 245, 247, 249, 250, 253, 254, 257, 258, 259, 262, 263, 267, 268, 271, 273, 275, 276, 277, 278, 281, 282, 283, 284, 286, 287, 288, 293, 298, 299, 300, 304, 306, 307]))
    NS=NS1[0:NUM_SYMBOLS]
    NUM_SYMBOLS = len(NS)


    all_freqs = pickle.load(open('market_cl_freqs.pkl','rb')) 

    for day in range(NUM_DAYS-1): # Last Day only for testing?
        task_accs = np.zeros((4, NUM_SYMBOLS, NUM_SYMBOLS)) 
        print(f'Begin Day {day}', priority = LogPriority.MEDIUM)
        task_num=0
        for task_id in NS:
            train_dl.dataset.set_day(day)
            train_dl.dataset.set_symbol(task_id)

            freqs = all_freqs[day][task_id]
            criterions = [None, None, None, None]
            for l in freqs:
                freq = freqs[l]
                print([v/(sum(freq.values())) for v in freq.values()])
                class_weights = [1/f if f != 0 else 100 for f in freq.values()]
                class_weights = torch.tensor([x/sum(class_weights) for x in class_weights], device = device)
                criterion = nn.CrossEntropyLoss(weight = class_weights)
                criterions[l] = criterion

            print('Frequencies for Current Day: ', freqs)
            # Step 1: First train the feature extractor on task 
            # '''
            import time
            start_time=time.time()
            print(f'Training Feature Extractor on task id {task_id}')
            loss, acc = train_fe(model, train_dl, criterions, task_id, device = device, lr = float(experimenter.config.TRAINING.LR.FE), num_epochs = experimenter.config.TRAINING.EPOCHS.FE)
            print(f'Feature Extraction Train Loss {loss} & Accuracy: {acc}')
            print("--- %s seconds ---" % (time.time() - start_time))
            
            print('\n---------------------------------------------------------\n')

            # Step 2: Now if task is 0, train the KB without Meta-Learning

            if baseline ==4:
                start_time = time.time()
                print(f'Training Initial Knowledge Base Weights')
                loss, acc = train_kb_nonmeta_baseline(model, train_dl, criterions, task_id,
                                             lr=float(experimenter.config.TRAINING.LR.INIT_KB), device=device,
                                             num_epochs=experimenter.config.TRAINING.EPOCHS.INIT_KB)
                print(f'Initial KB Validation Loss {loss} & Accuracy: {acc}')
                print("--- %s seconds ---" % (time.time() - start_time))

                print('\n---------------------------------------------------------\n')

            if (task_id == 0 and day==0 and baseline!=4) or baseline==1 or baseline==3:
                start_time = time.time()
                print(f'Training Initial Knowledge Base Weights')
                loss, acc = train_kb_nonmeta(model, train_dl, criterions, task_id, lr = float(experimenter.config.TRAINING.LR.INIT_KB), device = device, num_epochs = experimenter.config.TRAINING.EPOCHS.INIT_KB)
                print(f'Initial KB Validation Loss {loss} & Accuracy: {acc}')
                print("--- %s seconds ---" % (time.time() - start_time))

                print('\n---------------------------------------------------------\n')
            if baseline == 0 or baseline == 3:
                # Step 3: Now train MaskGenerator and Classifier
                start_time = time.time()
                print(f'Training Mask Generator and Classifier')
                loss, acc = train_mg_n_clf(model, train_dl, criterions, task_id, lr = float(experimenter.config.TRAINING.LR.MG_N_C), device = device, num_epochs = experimenter.config.TRAINING.EPOCHS.MG_N_C)
                print(f'Mask Generation and Classifier Training Loss {loss} & Accuracy: {acc}')
                print("--- %s seconds ---" % (time.time() - start_time))

                print('\n---------------------------------------------------------\n')

            if baseline == 0 or baseline == 2:
                # '''
                print('Updating KB')
                print('Before',sum([x.sum() for x in model.kb[0].parameters()]))
                start_time = time.time()
                update_kb(model, train_dl, train_dl, criterions, task_id, device=device)
                print(f'Update KB {loss} & Accuracy: {acc}')
                print('After',sum([x.sum() for x in model.kb[0].parameters()]))
                print("--- %s seconds ---" % (time.time() - start_time))

                print('\n---------------------------------------------------------\n')
                # '''

            if baseline == 0 or baseline == 3:
                # Stage 5: Fine-Tune Mask Generator and Final Classifier
                print(f'Fine-Tune Mask Generator and Classifier')
                start_time = time.time()
                train_mg_n_clf(model, train_dl, criterions, task_id, lr = float(experimenter.config.TRAINING.LR.FINETUNE_MG_N_C), num_epochs = experimenter.config.TRAINING.EPOCHS.FINETUNE_MG_N_C, device = device)
                print(f'Fine-Tuning Mask Generation and Classifier Training Loss {loss} & Accuracy: {acc}')
                print("--- %s seconds ---" % (time.time() - start_time))


                print('\n---------------------------------------------------------\n')

            # Needed only in case sup sup model is to be used.
            model.cache_masks()

            # Now report the numbers for task = 0 upto task = task_id
            start_time = time.time()
            print(f'Evaluating on task {task_id}\n', priority = LogPriority.MEDIUM)
            avg_loss = 0



            for t_id in range(task_id,task_id+1):
                train_dl.dataset.set_day(day+1)
                train_dl.dataset.set_symbol(t_id)
                loss, acc, f1s = test_loop(model, train_dl, t_id, criterions, device = device)
                print(f'Testing Loss for task = {t_id}: {loss}', priority = LogPriority.MEDIUM if task_id != experimenter.cfg.DATASET.NUM_TASKS-1 else LogPriority.STATS )
                print(f'Testing Accuracy for task = {t_id}: {acc}', priority = LogPriority.MEDIUM if task_id != experimenter.cfg.DATASET.NUM_TASKS-1 else LogPriority.STATS)
                print(f'Testing F1s for task = {t_id}: {f1s}\n', priority = LogPriority.MEDIUM if task_id != experimenter.cfg.DATASET.NUM_TASKS-1 else LogPriority.STATS)
                
                avg_loss += loss
                for i, ac in acc.items():
                    task_accs[i][task_num][task_num] = ac
            task_num=task_num+1
            print("--- %s seconds ---" % (time.time() - start_time))

        


        print(f'Day {day} complete', priority = LogPriority.STATS)

        # Now report the numbers for task = 0 upto task = task_id
        start_time = time.time()
        print(f'Evaluating on tasks {0} to {task_id}\n', priority = LogPriority.MEDIUM)
        avg_loss = 0
        t_num=task_num-1
        task_num=0
        for t_id in NS:
            # Set day and symbol for testing
            train_dl.dataset.set_day(day+1)
            train_dl.dataset.set_symbol(t_id)
            loss, acc, f1s = test_loop(model, train_dl, t_id, criterions, device = device)
            print(f'Testing Loss for task = {t_id}: {loss}', priority = LogPriority.MEDIUM if task_id != experimenter.cfg.DATASET.NUM_TASKS-1 else LogPriority.STATS )
            print(f'Testing Accuracy for task = {t_id}: {acc}', priority = LogPriority.MEDIUM if task_id != experimenter.cfg.DATASET.NUM_TASKS-1 else LogPriority.STATS)
            print(f'Testing F1s for task = {t_id}: {f1s}\n', priority = LogPriority.MEDIUM if task_id != experimenter.cfg.DATASET.NUM_TASKS-1 else LogPriority.STATS)
            
            avg_loss += loss
            for i, ac in acc.items():
                task_accs[i][t_num][task_num] = ac
            task_num=task_num+1
        print("--- %s seconds ---" % (time.time() - start_time))
        base_path = './'
        pathS = os.path.join(base_path,f'MARK_MODELS/MARKET/{baseline}_{iter}_model_{day}.pt')
        os.makedirs(os.path.split(pathS)[0], exist_ok=True)
        torch.save(model, pathS)

        for i in range(4):
            print(f'Label {i}', priority = LogPriority.STATS)
            print(task_accs[i])
            ta = task_accs[i]
            avg_acc = ta[-1].mean()
            bwt = sum(ta[-1]-np.diag(ta))/ (ta.shape[1]-1)
            print(f'Average Accuracy: {avg_acc}', priority = LogPriority.STATS)
            print(f'BWT: {bwt}', priority = LogPriority.STATS)
            days_accs[i][day] = avg_acc
            days_bwt[i][day] = bwt
    # Note this returns only for day 53 and 3rd label
    return days_accs.mean(axis=1), days_bwt.mean(axis=1)


In [ ]:
# Now train the model and print the accuracies and BWT
acc, bwt = train(model, criterion, train_dl, experimenter, device, baseline = 0, iter = 0)
print(f'Accuracies: {acc}', priority = LogPriority.STATS)
print(f'BWTS: {bwt}', priority = LogPriority.STATS)

# Contributors:
[Prerna Agarwal](https://github.com/prerna-agarwal-iitd)
<br>
[Pranjal Aggarwal](https://github.com/Pranjal2041/) 